In [124]:
import pandas as pd
import numpy as np
date='22-05-2024-with-bindings/'
target='gpu'


In [125]:
df_times = pd.concat([
  pd.read_csv(f'../data/{date}rust-{target}-times.csv'),
  pd.read_csv(f'../data/{date}python-{target}-times.csv'),
])
df_rewards = pd.concat([
  pd.read_csv(f'../data/{date}rust-{target}-rewards.csv'),
  pd.read_csv(f'../data/{date}python-{target}-rewards.csv'),
])
df = pd.concat([df_times, df_rewards], axis=1)

# Time and CPU

In [126]:
df.groupby('version')['elapsed_real_time_seconds'].describe()

,count,mean,std,min,25%,50%,75%,max
version,,,,,,,,
python,100.0,80.1399,19.957598,19.36,71.6625,82.71,90.3725,134.97
rust,100.0,58.3478,27.196359,18.59,38.0950,53.41,73.7950,160.20


In [127]:
df.groupby('version')['user_cpu_seconds'].describe()

,count,mean,std,min,25%,50%,75%,max
version,,,,,,,,
python,100.0,70.3251,19.788527,9.79,61.6675,72.870,80.4800,124.32
rust,100.0,49.2154,26.861636,9.68,29.1025,44.025,64.6025,149.30


In [128]:
df.groupby('version')['kernel_cpu_seconds'].describe()

,count,mean,std,min,25%,50%,75%,max
version,,,,,,,,
python,100.0,8.1697,0.154361,7.69,8.080,8.180,8.2725,8.50
rust,100.0,8.1038,0.312484,7.57,7.855,8.085,8.3125,9.12


In [129]:
df['cpu_percentage'] = df['cpu_percentage'].str[:-1].astype(int)
df.groupby('version')['cpu_percentage'].describe()

,count,mean,std,min,25%,50%,75%,max
version,,,,,,,,
python,100.0,97.23,1.354789,88.0,97.0,97.0,98.0,99.0
rust,100.0,97.35,1.366075,91.0,97.0,98.0,98.0,99.0


# Memory

'max_resident_set_size_kb',
'system_page_size_bytes',
'major_page_faults',
'minor_page_faults',

In [130]:
df.groupby('version')['max_resident_set_size_kb'].describe()

,count,mean,std,min,25%,50%,75%,max
version,,,,,,,,
python,100.0,2097651.64,12604.490578,2063808.0,2088794.0,2097448.0,2104237.0,2131124.0
rust,100.0,2002591.04,14828.189906,1977996.0,1993546.0,2000474.0,2008973.0,2081716.0


In [131]:
df.groupby('version')['major_page_faults'].describe()

,count,mean,std,min,25%,50%,75%,max
version,,,,,,,,
python,100.0,1.69,16.9,0.0,0.0,0.0,0.0,169.0
rust,100.0,1.25,12.5,0.0,0.0,0.0,0.0,125.0


In [132]:
df.groupby('version')['minor_page_faults'].describe()

,count,mean,std,min,25%,50%,75%,max
version,,,,,,,,
python,100.0,302775.32,12443.972518,284462.0,292009.50,297916.0,315909.75,327147.0
rust,100.0,275707.17,12080.154375,245785.0,266157.75,279997.0,284393.00,294786.0


# SO

'involuntary_context_switches',
'voluntary_context_switches', 
'filesystem_input_reads',
'filesystem_output_writes',

In [133]:
df.groupby('version')['involuntary_context_switches'].describe()

,count,mean,std,min,25%,50%,75%,max
version,,,,,,,,
python,100.0,196.81,35.108690,130.0,175.0,191.5,210.25,326.0
rust,100.0,128.84,22.624167,101.0,116.0,121.5,132.50,239.0


In [134]:
df.groupby('version')['voluntary_context_switches'].describe()

,count,mean,std,min,25%,50%,75%,max
version,,,,,,,,
python,100.0,35026.75,8405.381230,10008.0,31253.00,36284.5,40038.5,59234.0
rust,100.0,43437.28,23849.637944,11024.0,25319.75,39477.5,57655.0,140575.0


In [135]:
df.groupby('version')['filesystem_input_reads'].describe()

,count,mean,std,min,25%,50%,75%,max
version,,,,,,,,
python,100.0,901.04,9010.4,0.0,0.0,0.0,0.0,90104.0
rust,100.0,356.72,3567.2,0.0,0.0,0.0,0.0,35672.0


In [136]:
df.groupby('version')['filesystem_output_writes'].describe()

,count,mean,std,min,25%,50%,75%,max
version,,,,,,,,
python,100.0,16.0,0.0,16.0,16.0,16.0,16.0,16.0
rust,100.0,8.0,0.0,8.0,8.0,8.0,8.0,8.0


# Rewards

In [137]:
df.groupby('version')['training_steps'].describe()

,count,mean,std,min,25%,50%,75%,max
version,,,,,,,,
python,100.0,27140.0,7960.787738,2000.0,24000.0,28000.0,31250.0,50000.0
rust,100.0,10560.0,5908.938615,2000.0,6000.0,9500.0,14000.0,34000.0


In [138]:
df.groupby('version')['reward_mean'].describe()

,count,mean,std,min,25%,50%,75%,max
version,,,,,,,,
python,100.0,497.329,11.256740,402.700000,500.0,500.0,500.0,500.0
rust,100.0,498.156,6.759705,460.299988,500.0,500.0,500.0,500.0


In [139]:
df.groupby('version')['reward_std'].describe()

,count,mean,std,min,25%,50%,75%,max
version,,,,,,,,
python,100.0,4.814688,18.093653,0.0,0.0,0.0,0.0,120.085012
rust,100.0,4.893493,17.966263,0.0,0.0,0.0,0.0,93.900002


In [140]:
df['time/step'] = df['elapsed_real_time_seconds'] / df['training_steps']
df.groupby('version')['time/step'].describe()

,count,mean,std,min,25%,50%,75%,max
version,,,,,,,,
python,100.0,0.003095,0.000767,0.002499,0.002830,0.002932,0.003097,0.00968
rust,100.0,0.005911,0.000975,0.004712,0.005212,0.005738,0.006256,0.00970


In [141]:
# 	count	mean	std	min	25%	50%	75%	max
# version								
# python	100.0	0.003890	0.000649	0.003239	0.003589	0.003766	0.004094	0.009245
# rust	100.0	0.002552	0.000104	0.002339	0.002483	0.002533	0.002608	0.002864